# **C.O.L.L.A.B.**
<hr style="border:2px solid black">

## Goal: 
* 
* 
<hr style="border:2px solid black">

# Imports

In [12]:
import pandas as pd
import numpy as np
from wrangle import wrangle
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

* 

<hr style="border:2px solid black">

# **Prepare**

**Prepare Actions:**
* To better estimate family income based on family composition, use the estimates of family count by family type(Married, Single Male with kids, Single Male without kids, Single Female with kids, Single Female without kids) and the estimates of income by family type. e.g. (female_with_kids (income) * female_with_kids (count) + (male_with_kids (income) * male_with_kids (count)))/2
* Merge transformed census dataframe (family) on to the cost_of_living dataframe (col) on county.
* Adjust for inflation 2020 to 2022

In [13]:
cost, dash = wrangle()
cost

,county,parents,children,housing,food,transportation,healthcare,other,childcare,taxes,total,median_income
0,baldwin_county_al,1,0,9390,4048,11695,6096,4869,0,7035,43133,68231
1,baldwin_county_al,1,1,12058,5966,12558,9202,6530,6498,7734,60546,40290
2,baldwin_county_al,1,2,12058,8740,14025,12309,7536,16726,10329,81723,40290
3,baldwin_county_al,1,3,16334,11660,14661,15415,10143,19874,14165,102253,40290
4,baldwin_county_al,1,4,16334,14273,15010,18522,11090,19874,14640,109742,40290
...,...,...,...,...,...,...,...,...,...,...,...,...
8239,natrona_county_wy,2,0,9638,6488,14377,15581,5843,0,6990,58917,94332
8240,natrona_county_wy,2,1,11351,8075,15391,19471,7039,10372,8267,79966,103027
8241,natrona_county_wy,2,2,11351,10227,16148,23361,7819,22325,9462,100693,103027
8242,natrona_county_wy,2,3,16242,12487,16629,27251,10410,26003,11935,120957,103027


In [14]:
cost['affordability_ratio'] = round(cost.median_income / cost.total, 2)
cost

,county,parents,children,housing,food,transportation,healthcare,other,childcare,taxes,total,median_income,affordability_ratio
0,baldwin_county_al,1,0,9390,4048,11695,6096,4869,0,7035,43133,68231,1.58
1,baldwin_county_al,1,1,12058,5966,12558,9202,6530,6498,7734,60546,40290,0.67
2,baldwin_county_al,1,2,12058,8740,14025,12309,7536,16726,10329,81723,40290,0.49
3,baldwin_county_al,1,3,16334,11660,14661,15415,10143,19874,14165,102253,40290,0.39
4,baldwin_county_al,1,4,16334,14273,15010,18522,11090,19874,14640,109742,40290,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8239,natrona_county_wy,2,0,9638,6488,14377,15581,5843,0,6990,58917,94332,1.60
8240,natrona_county_wy,2,1,11351,8075,15391,19471,7039,10372,8267,79966,103027,1.29
8241,natrona_county_wy,2,2,11351,10227,16148,23361,7819,22325,9462,100693,103027,1.02
8242,natrona_county_wy,2,3,16242,12487,16629,27251,10410,26003,11935,120957,103027,0.85


<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

In [15]:
# create a dictionary to map the number of parents and children to a string
mapping = {(1, 0): 'single_no_child', (1, 1): 'single_1_child', (1, 2): 'single_2_child', (1, 3): 'single_3_child', (1, 4): 'single_4_child', (2, 0): 'married_no_child', (2, 1): 'married_1_child', (2, 2): 'married_2_child', (2, 3): 'married_3_child', (2, 4): 'married_4_child'}

# create a new column in the cost dataframe that maps the number of parents and children to a string
cost['family_type'] = cost.apply(lambda row: mapping[(row['parents'], row['children'])], axis=1)

# split the cost dataframe into ten dataframes based on the family_type column
family_types = ['single_no_child', 'single_1_child', 'single_2_child', 'single_3_child', 'single_4_child', 'married_no_child', 'married_1_child', 'married_2_child', 'married_3_child', 'married_4_child']
dfs = {}
for family_type in family_types:
    dfs[family_type] = cost[cost['family_type'] == family_type]


dataframes = {}
for status in ['single', 'married']:
    for i in range(5):
        key = f'{status}_{i}_child' if i != 0 else f'{status}_no_child'
        dataframes[key] = dfs[key]

In [ ]:
single_no_child = dataframes['single_no_child']
single_1_child = dataframes['single_1_child']
single_2_child = dataframes['single_2_child']
single_3_child = dataframes['single_3_child']


<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver


<hr style="border:2px solid black">

# **Comparing Models**


<hr style="border:2px solid black">

## **Best Model Evaluation on Test**

### <u>Modeling Summary:</u>

>* 

<hr style="border:2px solid black">

# **Conclusions**


### <u>Modeling:</u>

>**<u>:</u>**
>* 

### <u>Recommendations:</u>
>* 

### <u>Next Steps:</u>
>* 